In [106]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [107]:
import pymysql
import json
import warnings
import DB
warnings.filterwarnings(action='ignore')

In [108]:
db = DB.MYDB()
db.cursor.execute("select * from team_record where home_away = 'H' and gmkey >= 'S17' order by gmkey")
team_hdata = pd.DataFrame(db.cursor.fetchall())
team_hdata = team_hdata.drop(team_hdata[team_hdata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
team_hdata = team_hdata.reset_index(drop=True)
db.cursor.execute("select * from team_record where home_away = 'A' and gmkey >= 'S17' order by gmkey")
team_adata = pd.DataFrame(db.cursor.fetchall())
team_adata = team_adata.drop(team_adata[team_adata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
team_adata = team_adata.reset_index(drop=True)
db.db.close()


Connect!


In [109]:
def calculate_data(h_data, a_data):
  output_data = pd.DataFrame()
  output_data['gmkey'] = h_data['gmkey']
  output_data['ast'] = h_data['ast'] / h_data['fgt']
  # 2점
  output_data['twop'] = h_data['fg'] / h_data['fgA']
  # 3점
  output_data['threep'] = h_data['threep'] / h_data['threepA']
  # 자유투
  output_data['freep'] = h_data['ft'] / h_data['ftA']
  # 공격 리바
  output_data['offr'] = h_data['offr'] / (h_data['offr'] + a_data['defr'])
  # 속공
  # output_data['tfb'] = h_data['tfb'] / h_data['fgt']
  # 벤치득점
  # output_data['bench'] = h_data['benchScoreCn'] / h_data['score']
  # 속공득점
  # output_data['tfbp'] = h_data['fbScoreCn'] / h_data['score']
  # 페인트존 득점
  output_data['ppp'] = h_data['pp'] / h_data['ppA']
  # 덩크
  output_data['dk'] = h_data['dk'] / h_data['dkA']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = h_data['maxContiScoreCn'] / h_data['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (h_data['score'] - a_data['score']) / h_data['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = h_data['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = h_data['secChanceScoreCn'] / h_data['score']
  # 턴오버
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])

  output_data['defr'] = h_data['defr'] / (a_data['offr'] + h_data['defr'])
  output_data['stl'] = h_data['stl'] / (h_data['stl'] + a_data['stl'])
  output_data['bs'] = h_data['bs'] / a_data['fgtA']
  output_data['foul'] = h_data['foul'] / (h_data['foul'] + a_data['foul'])
  output_data['tf'] = h_data['tf'] / (h_data['tf'] + a_data['tf'])
  output_data['gd'] = h_data['gd'] / (h_data['gd'] + a_data['gd'])

  output_data['score'] = h_data['score']
  
  # output_data['teamR'] = h_data['teamR'] / (h_data['teamR'] + a_data['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data

In [110]:
def team_player_concat(team, player, team_rate):
  player_rate = 1 - team_rate
  output_data = pd.DataFrame()
  # output_data['gmkey'] = team['gmkey']
  output_data['ast'] = team_rate * team['ast'] + player_rate * player['ast']
  # 2점
  output_data['twop'] = team_rate * team['twop'] + player_rate * player['twop']
  # 3점
  output_data['threep'] = team_rate * team['threep'] + player_rate * player['threep']
  # 자유투
  output_data['freep'] = team_rate * team['freep'] + player_rate * player['freep']
  # 공격 리바
  output_data['offr'] = team_rate * team['offr'] + player_rate * player['offr']
  # 속공
  # output_data['tfb'] = team['tfb'] / team['fgt']
  # 벤치득점
  # output_data['bench'] = team['benchScoreCn'] / team['score']
  # 속공득점
  # output_data['tfbp'] = team['fbScoreCn'] / team['score']
  # 페인트존 득점
  output_data['ppp'] = team_rate * team['ppp'] + player_rate * player['ppp']
  # 덩크
  output_data['dk'] = team_rate * team['dk'] + player_rate * player['dk']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = team['maxContiScoreCn'] / team['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (team['score'] - player['score']) / team['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = team['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = team['secChanceScoreCn'] / team['score']
  # 턴오버
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])

  output_data['defr'] = team_rate * team['defr'] + player_rate * player['defr']
  output_data['stl'] = team_rate * team['stl'] + player_rate * player['stl']
  output_data['bs'] = team_rate * team['bs'] + player_rate * player['bs']
  output_data['foul'] = team_rate * team['foul'] + player_rate * player['foul']
  output_data['tf'] = team_rate * team['tf'] + player_rate * player['tf']
  output_data['gd'] = team_rate * team['gd'] + player_rate * player['gd']

  output_data['score'] = team['score']
  
  # output_data['teamR'] = team['teamR'] / (team['teamR'] + player['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data

In [111]:
def make_model(data):
    h_data = pd.DataFrame()
    a_data = pd.DataFrame()
    # 어시스트 -> 어시스트 / 필드골 성공
    h_data['ast'] = data['h_ast'] / data['h_fgt']
    a_data['ast'] = data['a_ast'] / data['a_fgt']
    # 2점
    h_data['twop'] = data['h_fg'] / data['h_fgA']
    a_data['twop'] = data['a_fg'] / data['a_fgA']

    # 3점
    h_data['threep'] = data['h_threep'] / data['h_threepA']
    a_data['threep'] = data['a_threep'] / data['a_threepA']

    # 자유투
    h_data['freep'] = data['h_ft'] / data['h_ftA']
    a_data['freep'] = data['a_ft'] / data['a_ftA']

    # 공격 리바
    h_data['offr'] = data['h_offr'] / (data['h_offr'] + data['a_defr'])
    a_data['offr'] = data['a_offr'] / (data['a_offr'] + data['h_defr'])
    # 속공
    h_data['tfb'] = data['h_tfb'] / data['h_fgt']
    a_data['tfb'] = data['a_tfb'] / data['a_fgt']

    # 벤치득점
    h_data['bench'] = data['h_benchScoreCn'] / data['h_score']
    a_data['bench'] = data['a_benchScoreCn'] / data['a_score']


    # 속공득점
    h_data['tfbp'] = data['h_fbScoreCn'] / data['h_score']
    a_data['tfbp'] = data['a_fbScoreCn'] / data['a_score']

    # 페인트존 득점
    h_data['ppp'] = data['h_pp'] / data['h_ppA']
    a_data['ppp'] = data['a_pp'] / data['a_ppA']

    # 덩크
    h_data['dk'] = data['h_dk'] / data['h_dkA']
    a_data['dk'] = data['a_dk'] / data['a_dkA']

    # 최다 연속 득점 비율
    h_data['maxContiScore'] = data['h_maxContiScoreCn'] / data['h_score']
    a_data['maxContiScore'] = data['a_maxContiScoreCn'] / data['a_score']

    # 최다 리드 점수차
    # h_data['maxLeadScore'] = (data['h_score'] - data['a_score']) / data['h_maxLeadScoreCn']
    # a_data['maxLeadScore'] = (data['a_score'] - data['h_score']) / data['a_maxLeadScoreCn']
    # h_data[h_data['maxLeadScore'] < -1]

    # 득점 우위 시간
    h_data['scoreHighTime'] = data['h_scoreHighTimeCn'] / 2400
    a_data['scoreHighTime'] = data['a_scoreHighTimeCn'] / 2400

    # 세컨찬스 득점
    h_data['secChanceScore'] = data['h_secChanceScoreCn'] / data['h_score']
    a_data['secChanceScore'] = data['a_secChanceScoreCn'] / data['a_score']

    # 턴오버
    h_data['to'] = (data['h_to'] + data['h_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])
    a_data['to'] = (data['a_to'] + data['a_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])

    h_data['defr'] = data['h_defr'] / (data['a_offr'] + data['h_defr'])
    a_data['defr'] = data['a_defr'] / (data['h_offr'] + data['a_defr'])
    h_data['stl'] = data['h_stl'] / (data['h_stl'] + data['a_stl'])
    a_data['stl'] = data['a_stl'] / (data['h_stl'] + data['a_stl'])
    h_data['bs'] = data['h_bs'] / data['a_fgtA']
    a_data['bs'] = data['a_bs'] / data['h_fgtA']
    h_data['foul'] = data['h_foul'] / (data['h_foul'] + data['a_foul'])
    a_data['foul'] = data['a_foul'] / (data['h_foul'] + data['a_foul'])
    h_data['tf'] = data['h_tf'] / (data['h_tf'] + data['a_tf'])
    a_data['tf'] = data['a_tf'] / (data['h_tf'] + data['a_tf'])
    h_data['gd'] = data['h_gd'] / (data['h_gd'] + data['a_gd'])
    a_data['gd'] = data['a_gd'] / (data['h_gd'] + data['a_gd'])
    h_data['teamR'] = data['h_teamR'] / (data['h_teamR'] + data['a_teamR'])
    a_data['teamR'] = data['a_teamR'] / (data['h_teamR'] + data['a_teamR'])

    # h_data = h_data.fillna(0)
    # h_data = h_data.replace([np.inf, -np.inf], 0)
    h_data['score'] = data['h_score']
    # h_data['gmkey'] = data['gmkey']

    # a_data = a_data.fillna(0)
    # a_data = a_data.replace([np.inf, -np.inf], 0)
    a_data['score'] = data['a_score']
    # a_data['gmkey'] = data['gmkey']
    return pd.concat([h_data, a_data], ignore_index=True)

In [112]:
#seasonKey Example : S19G01
def get_keyplayer(seasonKey, team):
    result = set()
    season = str(int(seasonKey[1:3]) -2)
    league = seasonKey[4:6]
    seasonKey = 'S' + season + 'G' + league
    db.cursor.execute('''SELECT *
                        FROM player_avg_record
                        WHERE seasonCode LIKE 'S{}%' and
                        pcode IN (SELECT pcode
                                    FROM player
                                    WHERE tcode = '{}' and seasonCode = '{}');'''.format(season, team, season))
    data = pd.DataFrame(db.cursor.fetchall())
    if data.size != 0:
        score_top_player = data.sort_values('score',ascending=False)['pcode'][:3]
        ast_top_player = data.sort_values('ast')['pcode'][:3]
        rb_top_player = data.sort_values('rb')['pcode'][:3]
        for player in score_top_player:
            result.add(player)
        # result.add(score_top_player[0])
        for player in ast_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        for player in rb_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        return result     
    else:
        return None

In [113]:
def get_season_keyplayer(season):
    # db = DB.MYDB()
    result = dict()
    seasonCode = 'S' + str(season) + 'G01'
    for tcode in tcodes:
        tcode = str(tcode['tcode'])
        # if tcode == '75':
            # tcode = '29'
        # db.cursor.execute('''SELECT *
        #                 FROM player_avg_record
        #                 WHERE seasonCode LIKE 'S{}%' and
        #                 pcode IN (SELECT pcode
        #                             FROM player
        #                             WHERE tcode = '{}' and seasonCode = '{}');'''.format(season - 2, tcode, season))
        db.cursor.execute('''SELECT *
                        FROM (
                            SELECT p.*, rank() over(partition by p.pcode order by p.seasonCode desc) as a
                            FROM player_avg_record as p
                            WHERE seasonCode < 'S{}' and
                                pcode IN (SELECT pcode
                                            FROM player
                                            WHERE tcode = '{}' and seasonCode = '{}')
                            ) as rankrow
                        where rankrow.a <=1;
        '''.format(seasonCode, tcode, season))
        data = pd.DataFrame(db.cursor.fetchall())
        if data.size != 0:
            score_top_player = data.sort_values('score',ascending=False)['pcode'][:1]
            ast_top_player = data.sort_values('ast', ascending=False)['pcode'][:1]
            rb_top_player = data.sort_values('rb', ascending=False)['pcode'][:1]
            result[tcode] = set(score_top_player) | set(ast_top_player) | set(rb_top_player)
    return result

In [114]:
def get_recent_players(players, gameDate, db):
    # df = pd.DataFrame()
    df = []
    # db = DB.MYDB()
    # for player in players:
        # db.cursor.execute("""
        #                 SELECT b.*, a.gameDate 
        #                 FROM game_meta a, player_record b
        #                 WHERE a.gmkey = b.gmkey and b.pcode = '{}' and b.gmkey < '{}'
        #                 ORDER BY gameDate desc
        #                 LIMIT 5""".format(player, gameCode))
    if len(players) == 1:
        players.add('0')
    db.cursor.execute("""
                        select *
                        from (
                            SELECT b.*, a.gameDate, rank() over(partition by b.pcode order by gameDate desc) as a
                            FROM game_meta a, player_record b
                            WHERE a.gmkey = b.gmkey and a.gameDate < '{}'
                                and b.pcode in {}
                            ) as rankrow
                        where rankrow.a <=5;""".format(gameDate, tuple(players)))
    # db.db.close()
    return pd.DataFrame(db.cursor.fetchall())


In [115]:
def get_keyplayer_game_record():
    db = DB.MYDB()
    df = pd.DataFrame()
    for season in season_keyplayer:
        for tcode in season_keyplayer[season]:
            pcode = season_keyplayer[season][tcode]
            if len(pcode) == 1:
                pcode.add('0')
            try:
            
                db.cursor.execute("""
                                SELECT * FROM hygp.player_record a join game_meta b on a.gmkey = b.gmkey
                                where (b.tcodeH = '{}' or b.tcodeA = '{}') and a.gmkey >= 'S17'
                                and a.pcode in {};
                                """.format(tcode, tcode, tuple(pcode)))
                data = pd.DataFrame(db.cursor.fetchall())
                df = pd.concat([df, data.groupby('gmkey').mean()], ignore_index=True)
            except Exception as e:
                print(season, tcode)
                print(pcode)
                print(data)
                input()
            # print(df)
        
    db.db.close()

    return df

In [116]:
db = DB.MYDB()
db.cursor.execute("SELECT gmkey, tcodeA, tcodeH, gameDate FROM hygp.game_meta where gmkey >= 'S17G01N1' order by gmkey;")
records = db.cursor.fetchall()
db.cursor.execute("SELECT gmkey, score, loss FROM hygp.team_record where home_away = 'H' and gmkey >= 'S17' order by gmkey;")
scores = db.cursor.fetchall()
db.cursor.execute("select tcode from team_code")
tcodes = db.cursor.fetchall()
tcodes = [tcode['tcode'] for tcode in tcodes]
# season_keyplayer = dict()
# for season in range(17, 41):
#     season_keyplayer[str(season)] = get_season_keyplayer(season)
db.db.close()
    

Connect!


In [125]:
def get_team_player(db):
    db.cursor.execute("select pcode, tcode, seasonCode from player;")
    total_players = pd.DataFrame(db.cursor.fetchall())
    players = dict()
    for season in range(17, 41):
        season_player = total_players[total_players['seasonCode'] == str(season)]
        players[str(season)] = dict()
        for tcode in tcodes: 
            players[str(season)][tcode] = tuple(season_player[season_player['tcode'] == tcode]['pcode'])
    return players


In [137]:
def get_recent_keyplayer(player, gameDate, db):
    db.cursor.execute("""
                    select *
                    from (
                        SELECT r.pcode, r.trueShooting, r.astRatio, r.rebRatio, m.gameDate, rank() over(partition by r.pcode order by gameDate desc) as a
                        FROM player_record r join game_meta m
                        WHERE r.gmkey = m.gmkey and m.gameDate < '{}'
                            and r.pcode in {}
                            and (r.playMin != 0 and r.playSec != 0)
                        ) as rankrow
                    where rankrow.a <=5;
                        """.format(gameDate, player))
    data = pd.DataFrame(db.cursor.fetchall()).groupby('pcode').mean()
    top_shoot = data.sort_values('trueShooting', ascending=False)[:1].index
    top_ast = data.sort_values('astRatio', ascending=False)[:1].index
    top_reb = data.sort_values('rebRatio', ascending=False)[:1].index

    top_player = set(top_shoot) | set(top_ast) | set(top_reb)

    return top_player


    

In [168]:
def get_keyplayer_data():
    db = DB.MYDB()
    db.cursor.execute("select * from game_meta where gameDate >= '20101015'")
    meta_data = db.cursor.fetchall()
    players = get_team_player(db)
    df = pd.DataFrame()
    for meta in meta_data[]:
        gmKey = meta['gmkey']
        gameDate = meta['gameDate']
        season = gmKey[1:3]
        a_player = players[season][meta['tcodeA']]
        if meta['tcodeA'] == '75':
            a_player = players[str(int(season)+1)]['29']
        h_player = players[season][meta['tcodeH']]
        if meta['tcodeH'] == '75':
            h_player = players[str(int(season)+1)]['29']

        key_player =  tuple(get_recent_keyplayer(h_player, gameDate, db) | get_recent_keyplayer(a_player, gameDate, db))

        db.cursor.execute("""
                    select avg(ast) as ast, avg(bs) as bs, avg(defr) as defr, avg(dk) as dk, avg(dkA) as dkA, avg(ef) as ef, 
                            avg(fb) as fb, avg(fg) as fg, avg(fgA) as fgA, avg(fgtA) as fgtA, avg(fo) as fo, avg(foul) as foul, 
                            avg(ft) as ft, avg(ftA) as ftA, avg(gd) as gd, avg(offr) as offr, avg(pp) as pp, avg(ppA) as ppA, avg(pscore) as pscore, 
                            avg(stl) as stl, avg(tf) as tf, avg(threep) as threep, avg(threepA) as threepA, avg(r.to) as turonover, avg(wft) as wft, 
                            avg(woft) as woft, avg(rb) as rb, avg(score) as score, avg(trueShooting) as trueShooting, avg(astRatio) as astRatio, avg(rebRatio) as rebRatio,
                            home_away
                            from player_record r
                            where gmKey = '{}' and pcode in {}
                            group by home_away
        """.format(gmKey, key_player))
        data = pd.DataFrame(db.cursor.fetchall())
        data['gmKey'] = gmKey
        df = pd.concat([df, data], ignore_index=True)
        print(gmKey)
    df.to_csv('key_player_recent5.csv')
    db.db.close()

    

Connect!
S37G13N2
S37G13N3
S37G13N4
S37G13N5
S37G13N6
S37G13N7
S37G13N8
S37G13N9
S37G13N10
S37G13N11
S37G13N12
S37G13N13
S37G13N14
S38G01N1
S38G01N2
S38G01N3
S38G01N4
S38G01N5
S38G01N6
S38G01N7
S38G01N8
S38G01N9
S38G01N10
S38G01N11
S38G01N12
S38G01N13
S38G01N14
S38G01N15
S38G01N16
S38G01N17
S38G01N18
S38G01N19
S38G01N20
S38G03N1
S38G03N2
S38G03N3
S38G03N4
S38G04N1
S38G04N2
S38G08N1
S38G08N2
S38G08N3
S38G08N4
S38G08N5
S38G08N6
S38G08N7
S38G08N8
S38G08N9
S38G08N10
S38G08N11
S38G08N12
S38G08N13
S38G08N14
S38G08N15
S38G08N16
S38G08N17
S38G08N18
S38G08N19
S38G08N20
S38G08N21
S39G01N1
S39G01N2
S39G01N3
S39G01N4
S39G01N5
S39G01N6
S39G01N7
S39G01N8
S39G01N9
S39G01N10
S39G01N11
S39G01N12
S39G01N13
S39G01N14
S39G01N15
S39G01N16
S39G01N17
S39G01N18
S39G01N19
S39G01N20
S39G01N21
S39G01N22
S39G01N23
S39G01N24
S39G01N25
S39G01N26
S39G01N27
S39G01N28
S39G01N29
S39G01N30
S39G01N31
S39G01N32
S39G01N33
S39G01N34
S39G01N35
S39G01N36
S39G01N37
S39G01N38
S39G01N39
S39G01N40
S39G01N41
S39G01N42
S39G01N43
S3

In [170]:
pd.read_csv("key_player_recent5_total.csv", index_col=0)

,ast,bs,defr,dk,dkA,ef,fb,fg,fgA,fgtA,...,turonover,wft,woft,rb,score,trueShooting,astRatio,rebRatio,home_away,gmKey
0,2.3333,0.0000,1.3333,0.0,0.0,0.0,0.0,2.6667,3.3333,4.3333,...,1.3333,0.3333,0.6667,1.3333,8.0000,0.255754,8.758759,83.753087,1,S17G01N1
1,1.0000,0.6667,0.6667,0.0,0.0,0.0,0.0,0.6667,2.0000,2.6667,...,1.6667,0.0000,0.6667,0.6667,4.3333,0.203634,5.364807,43.960602,2,S17G01N1
2,5.0000,0.0000,4.0000,0.0,0.0,0.0,0.0,2.0000,4.0000,10.0000,...,6.0000,3.0000,1.0000,4.0000,14.0000,0.643382,22.851919,257.247253,1,S17G01N2
3,1.5000,0.0000,0.5000,0.0,0.0,0.0,0.0,0.0000,1.5000,3.0000,...,1.0000,0.5000,0.5000,0.5000,3.5000,0.271739,13.111888,38.465275,2,S17G01N2
4,3.0000,0.0000,5.0000,0.0,0.0,0.0,0.0,2.0000,4.0000,6.0000,...,3.0000,2.0000,0.0000,7.0000,10.0000,0.525210,19.329897,581.180786,1,S17G01N3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,3.5000,0.0000,2.5000,0.0,0.0,0.0,0.0,2.5000,4.5000,8.5000,...,0.5000,2.0000,0.0000,4.5000,8.0000,0.545349,16.859344,434.315483,1,S40G01N36
682,3.0000,0.0000,2.5000,0.0,0.0,0.0,0.0,3.0000,4.5000,12.0000,...,1.5000,0.5000,1.0000,4.5000,12.5000,0.475646,15.107415,429.000000,2,S40G01N37
683,2.5000,0.0000,2.0000,0.0,0.0,0.0,0.0,5.5000,7.0000,9.0000,...,0.5000,1.0000,1.0000,3.5000,15.0000,0.878397,10.096931,479.444977,1,S40G01N37
684,0.0000,1.0000,4.0000,0.0,0.0,0.0,0.0,5.0000,7.0000,7.0000,...,0.0000,2.0000,1.0000,9.0000,12.0000,0.684932,0.000000,841.916931,2,S40G01N38


In [119]:
def get_recent_team(team, gameDate, db):
    db.cursor.execute("""select b.*, a.gameDate 
                        from game_meta a, team_record b
                        where a.gmkey = b.gmkey and a.gameDate <'{}'
                            and b.tcode = '{}'
                        order by a.gameDate desc
                        limit 5""".format(gameDate, team))
    # db.db.close()
    return pd.DataFrame(db.cursor.fetchall())


In [120]:
def make_player_recent_data():
    h_data = []
    a_data = []
    db = DB.MYDB()
    for idx, (record, score) in enumerate(zip(records, scores)):
        gmkey = record['gmkey']
        gameDate = record['gameDate']
        season = gmkey[1:3]
        h_score = score['score']
        a_score = score['loss']
        tcodeH = record['tcodeH']
        tcodeA = record['tcodeA']
        if gmkey != score['gmkey']:
            print('index error', gmkey, score['gmkey'])
        if tcodeH == '75' or tcodeA == '75': #예외처리
            continue
        try:
            temp1 = dict(get_recent_players(season_keyplayer[season][tcodeH], gameDate, db).mean(numeric_only=True))
            temp1['score'] = h_score
            temp1['gmkey'] = gmkey
            temp2 = dict(get_recent_players(season_keyplayer[season][tcodeA], gameDate, db).mean(numeric_only=True))
            temp2['score'] = a_score
            temp2['gmkey'] = gmkey

            h_data.append(temp1)
            a_data.append(temp2)
        except Exception as e:
            print("error ", gmkey)
        if idx % 100 == 0:
            print(gmkey)
        

    h_df = pd.DataFrame(h_data).to_csv("h_data.csv")
    a_df = pd.DataFrame(h_data).to_csv("a_data.csv")
    # model = pd.concat([make_data(h_data, a_data), make_data(a_data, h_data)])
    # return model
    db.db.close()


In [121]:
def make_team_recent_data():
    h_data = []
    a_data = []
    db = DB.MYDB()
    for idx, (record, score) in enumerate(zip(records, scores)):
        gmkey = record['gmkey']
        gameDate = record['gameDate']
        h_score = score['score']
        a_score = score['loss']
        if gmkey != score['gmkey']:
            print('index error', gmkey, score['gmkey'])
        if record['tcodeH'] == '75' or record['tcodeA'] == '75': #예외처리
            continue
        try:
            temp1 = dict(get_recent_team(record['tcodeH'], gameDate, db).mean(numeric_only=True))
            temp1['score'] = h_score
            temp1['gmkey'] = gmkey
            temp2 = dict(get_recent_team(record['tcodeA'], gameDate, db).mean(numeric_only=True))
            temp2['score'] = a_score
            temp2['gmkey'] = gmkey

            h_data.append(temp1)
            a_data.append(temp2)
        except Exception as e:
            print("error ", gmkey)
            e.with_traceback()
        if idx % 100 == 0:
            print(gmkey)
        

    h_df = pd.DataFrame(h_data).to_csv("team_h_data_recent5.csv")
    a_df = pd.DataFrame(h_data).to_csv("team_a_data_recent5.csv")
    # model = pd.concat([make_data(h_data, a_data), make_data(a_data, h_data)])
    # return model
    db.db.close()

In [166]:
'3' - 1

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [167]:
player_hdata = pd.read_csv('h_data.csv', index_col=0)
player_adata = pd.read_csv('a_data.csv', index_col=0)
# result = calculate_data(h_data,a_data)
# player_data = pd.concat([calculate_data(h_data,a_data), calculate_data(a_data,h_data)], ignore_index=True)

In [202]:
player_hdata.sort_values(by='gmkey', inplace=True)
player_adata.sort_values(by='gmkey', inplace=True)
team_hdata.sort_values(by='gmkey', inplace=True)
team_adata.sort_values(by='gmkey', inplace=True)

In [221]:
home_result = team_player_concat(calculate_data(team_hdata,team_adata), calculate_data(player_hdata, player_adata), 0)
away_result = team_player_concat(calculate_data(team_adata,team_hdata), calculate_data(player_adata, player_hdata), 0)
result = pd.concat([home_result, away_result], ignore_index=True)
result

,gmkey,ast,twop,threep,freep,offr,ppp,dk,defr,stl,bs,foul,tf,gd,score
0,S17G01N1,0.683333,0.545455,0.315789,0.771429,0.338462,0.659091,0.0,0.661538,0.5,0.029851,0.5,NaN,0.5,99
1,S17G01N10,0.746479,0.480392,0.366667,0.757576,0.154930,0.486111,1.0,0.845070,0.5,0.037037,0.5,NaN,0.5,90
2,S17G01N100,0.454545,0.477064,0.350000,0.829787,0.434211,0.559322,NaN,0.565789,0.5,0.046980,0.5,NaN,NaN,75
3,S17G01N101,0.863636,0.507463,0.263158,0.647059,0.147541,0.555556,NaN,0.852459,0.5,0.057143,0.5,NaN,NaN,92
4,S17G01N102,0.454545,0.612245,0.428571,0.754098,0.318584,0.684685,1.0,0.681416,0.5,0.119048,0.5,NaN,NaN,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7827,S40G01N5,0.230769,0.368421,0.375000,0.500000,0.205882,0.545455,NaN,0.794118,0.5,0.085714,0.5,NaN,NaN,72
7828,S40G01N6,1.050000,0.460000,0.447368,0.812500,0.040000,0.515152,NaN,0.960000,0.5,0.000000,0.5,NaN,0.5,80
7829,S40G01N7,1.109375,0.611765,0.413793,0.388889,0.285714,0.676471,NaN,0.714286,0.5,0.035088,0.5,NaN,NaN,77
7830,S40G01N8,0.388889,0.333333,0.350000,0.400000,0.385965,0.416667,NaN,0.614035,0.5,0.094340,0.5,NaN,NaN,92


In [222]:
# data1 = result.replace(0, 0.1)
data1 = result.fillna(0)
data1 = data1.drop(columns='gmkey')

In [223]:
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [225]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [226]:
y_predict = model.predict(X_test)
mean_squared_error(y_predict, y_test, squared = False)

11.464793688620365

In [227]:
model.score(X_test, y_test)


0.03651125142163203

In [59]:
tuple({'1234'})

('1234',)